In [1]:
import pandas as pd

In [2]:
import os

In [3]:
import numpy as np

In [4]:
path = os.path.join("Resources", "purchase_data.csv")

In [5]:
df = pd.read_csv(path)

In [6]:
#display the total count of players

In [7]:
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [8]:
tot_players_df = pd.DataFrame ([df["SN"].nunique()],columns=["Total Players"])

# Player Count

Display the total number of players

In [9]:
tot_players_df

,Total Players
0,576


#Run basic calculations to obtain number of unique items, average price, etc.

In [10]:
# tot_players = number of unique players

In [11]:
tot_players = sum(tot_players_df["Total Players"])

In [12]:
summary = [df["Item ID"].nunique(),\
           df["Price"].mean(),\
           df["Purchase ID"].count(),\
           df["Price"].sum()]

In [13]:
# create df2 to summarize 

In [14]:
df2 = pd.DataFrame([summary], columns=["Number of Unique Items", 
                                       "Average Price", 
                                       "Number of Purchases", 
                                       "Total Revenue"])

    Create format dict to format dataframe columns with correct $,%, and decimal places
    This has advantages because it displays in the correct format but DOES NOT mutate the dataframe
    https://pbpython.com/styling-pandas.html

In [15]:
format_dict = {"Average Price":"${:.2f}", 
               "Total Revenue": "${:.2f}", 
               "Percentage of Players":"{:.2%}",
               "Average Purchase Price":"${:.2f}",
               "Total Purchase Value":"${:.2f}",
               "Avg Total Purchase per Person":"${:.2f}",
              }

# Purchasing Analysis (Total)

In [16]:
df2.style.format(format_dict)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


#### Perform Gender Demographics
    Percentage and Count of Male Players
    Percentage and Count of Female Players
    Percentage and Count of Other / Non-Disclosed

In [17]:
gend_df = df.loc[:,["Gender", "SN"]].drop_duplicates()

In [18]:
gend_df = gend_df.groupby(["Gender"])\
    .size().reset_index(name="Total Count")\
        .sort_values(by=['Total Count'], ascending=False)

In [19]:
gend_df["Percentage of Players"] = gend_df["Total Count"].apply(lambda x: x/tot_players)

## Gender Demographics
   

In [20]:
gend_df.style.format(format_dict)

,Gender,Total Count,Percentage of Players
1,Male,484,84.03%
0,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


#### Perfrom Purchaseing Analysis (Gender)
    Run basic calculations to obtain purchase count, 
    avg. purchase price, avg. purchase total per person etc. by gender
    Create a summary data frame to hold the results

In [21]:
df3 = df.groupby("Gender").agg(Purchase_Count = ("Gender", "count"),
                                  Average_Purchase_Price = ("Price","mean"),
                                  Total_Purchase_Value=("Price","sum"))


In [22]:
df3 = pd.merge(left=df3,right=gend_df, 
               how='left', 
               left_on="Gender", 
               right_on="Gender").drop(["Percentage of Players"], axis = 1)

In [23]:
df3["Avg Total Purchase per Person"] = df3["Total_Purchase_Value"]/df3["Total Count"]

In [24]:
df3 = df3.drop(["Total Count"], axis =1)

In [25]:
df3.columns = ["Gender", 
               "Purchase Count", 
               "Average Purchase Price", 
               "Total Purchase Value", 
               "Avg Total Purchase per Person"]

## Purchasing Analysis (Gender)

In [26]:
df3.style.format(format_dict)

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,$1967.64,$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


#### Perform Age Demographics
    Establish bins for ages
    Categorize the existing players using the age bins. 
    Calculate the numbers and percentages by age group
    Create a summary data frame to hold the results
    Optional: round the percentage column to two decimal points
    Display Age Demographics Table

In [27]:
df4 = df

In [28]:
df4['Age Ranges'] = pd.cut(df4['Age'],[0, 9, 14, 19, 24, 29, 34, 39, 1000],\
                   labels=["<10", 
                        "10-14", 
                        "15-19", 
                        "20-24",
                        "25-29",
                        "30-34", 
                        "35-39",
                        "40+"])

In [29]:
age_binned = df4.loc[:,["Age Ranges", "SN"]].drop_duplicates()

In [30]:
age_binned = age_binned.groupby(["Age Ranges"]).size().reset_index(name="Total Count")

In [31]:
age_binned["Percentage of Players"] = age_binned["Total Count"].apply(lambda x: x/tot_players)

## Age Demographics

In [32]:
age_binned.style.format(format_dict)

,Age Ranges,Total Count,Percentage of Players
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


#### Purchasing Analysis (Age)
    Bin the purchase_data data frame by age
    Run basic calculations to obtain purchase count, avg. 
    purchase price, avg. purchase total per person etc. in the table below
    Create a summary data frame to hold the results
    Optional: give the displayed data cleaner formatting
    Display the summary data frame

In [33]:
df4 = df4.groupby("Age Ranges").agg(Purchase_Count = ("Age Ranges", "count"),
                                  Average_Purchase_Price = ("Price","mean"),
                                  Total_Purchase_Value=("Price","sum"))

In [34]:
df4 = pd.merge(left=df4,right=age_binned, 
               how='left', 
               left_on="Age Ranges", 
               right_on="Age Ranges").drop(["Percentage of Players"], axis = 1)

In [35]:
df4["Avg Total Purchase per Person"] = df4["Total_Purchase_Value"]/df4["Total Count"]

In [36]:
df4 = df4.drop(["Total Count"], axis =1)

In [37]:
df4.columns = ["Age Ranges", 
               "Purchase Count", 
               "Average Purchase Price", 
               "Total Purchase Value", 
               "Avg Total Purchase per Person"]

## Purchasing Analysis (Age)

In [38]:
df4.style.format(format_dict)

,Age Ranges,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,$1114.06,$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


#### Top Spenders
    Run basic calculations to obtain the results in the table below
    Create a summary data frame to hold the results
    Sort the total purchase value column in descending order
    Optional: give the displayed data cleaner formatting
    Display a preview of the summary data frame

In [39]:
df5 = df.groupby("SN").agg(Purchase_Count = ("SN", "count"),
                                  Average_Purchase_Price = ("Price","mean"),
                                  Total_Purchase_Value=("Price","sum"))

In [40]:
df5.columns = ["Purchase Count", 
               "Average Purchase Price", 
               "Total Purchase Value"]

In [41]:
df5 = df5.sort_values(by=["Total Purchase Value"], ascending=False)

In [42]:
df5["Average Purchase Price"] = df5["Average Purchase Price"].map('${:,.2f}'.format)

In [43]:
df5["Total Purchase Value"] = df5["Total Purchase Value"].map('${:,.2f}'.format)

In [44]:
# I would have used the style format_dict method here but you cannot use head()

## Top Spenders

In [45]:
df5.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


#### Most Popular Items
    Retrieve the Item ID, Item Name, and Item Price columns
    Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
    Create a summary data frame to hold the results
    Sort the purchase count column in descending order
    Optional: give the displayed data cleaner formatting
    Display a preview of the summary data fram

In [46]:
item_df = df.loc[:,["Item ID", "Item Name"]].drop_duplicates()

In [47]:
df6 = df.groupby("Item ID").agg(Purchase_Count = ("Item ID", "count"),
                                  Average_Purchase_Price = ("Price","mean"),
                                  Total_Purchase_Value=("Price","sum"))

In [48]:
df6.columns = ["Purchase Count", 
               "Average Purchase Price", 
               "Total Purchase Value"]

In [49]:
df6 = pd.merge(left=item_df,right=df6, 
               how='outer', 
               left_on="Item ID", 
               right_on="Item ID")

In [50]:
df7 = df6.sort_values(by=["Purchase Count"], ascending=False)

In [51]:
df7["Average Purchase Price"] = df7["Average Purchase Price"].map('${:,.2f}'.format)

In [52]:
df7["Total Purchase Value"] = df7["Total Purchase Value"].map('${:,.2f}'.format)

## Most Popular Items

In [53]:
df7.head()

,Item ID,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
0,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
17,82,Nirvana,9,$4.90,$44.10
94,145,Fiery Glass Crusader,9,$4.58,$41.22
134,19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


#### Most Profitable Items
    Sort the above table by total purchase value in descending order
    Optional: give the displayed data cleaner formatting
    Display a preview of the data frame

In [54]:
df8 = df6.sort_values(by=["Total Purchase Value"], ascending=False)

In [55]:
df8["Average Purchase Price"] = df8["Average Purchase Price"].map('${:,.2f}'.format)

In [56]:
df8["Total Purchase Value"] = df8["Total Purchase Value"].map('${:,.2f}'.format)

## Most Profitable Items

In [57]:
df8.head()

,Item ID,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
17,82,Nirvana,9,$4.90,$44.10
94,145,Fiery Glass Crusader,9,$4.58,$41.22
2,92,Final Critic,8,$4.88,$39.04
112,103,Singed Scalpel,8,$4.35,$34.80


## Observations:
    The age range of 2-24 is the both the biggest user of the game
    The age group of 35-39 is the biggest spender per person
    Oathbreaker, Last Hope of the Breaking storm is the most popular Item
    The demographic where the game is most popular is 20-24 year olds and primarily males (~8:1 M:F)